You can follow along and play with this notebook by clicking the badge below

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jasongfleischer/UCSD_COGS118A/blob/main/Notebooks/Lecture_03_linear_regression.ipynb)

## Linear regression


In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression



# some utility functions to create toy data
# data ~ underlying function + gaussian noise

def create_toy_data(func, sample_size, std):
    x = np.linspace(0, 1, sample_size).reshape(-1, 1)
    t = func(x) + np.random.normal(scale=std, size=x.shape)
    return x, t

def a_sinusoidal_func(x):
    return np.sin(2 * np.pi * x)

def a_polynomial_func(x):
    return (12. + 6.14*x - 8.4*x*x)

def an_exp_func(x):
    return (1+1*np.exp(0.001*x))

def a_linear_func(x):
    return (1.17 + 3.14*x)

def a_discontinuous_func(x):
    return [ 1. if el>0.5 else 0. for el in x  ]
    
sample_size = 10
sigma = 0.3

#func = a_sinusoidal_func
#func = a_polynomial_func
#func = a_discontinuous_func
func = a_linear_func

# Some toy data generated by a linear function + noise

In [ ]:
np.random.seed(1234)
x_train, y_train = create_toy_data(func, sample_size, sigma)
x_predict = np.linspace(0, 1, 100).reshape(-1, 1) #  some x-vals so we can generate true y-vals
y_true = func(x_predict)


plt.scatter(x_train, y_train, facecolor="none", edgecolor="b", s=50, label="training data")
plt.plot(x_predict, y_true, c="g", label="generating function")
plt.ylim(bottom=0.0)
plt.legend()
plt.show()

# Polynomial features:   $ 1 + x + x^2 \ldots$ 

In [ ]:
feature = PolynomialFeatures(degree=1)
feature.fit_transform(x_train)

In [ ]:
feature = PolynomialFeatures(degree=2)
feature.fit_transform(x_train)

# Solving for $\mathbf{w}$ using closed-form solution of OLS

![math eqn of psuedoinverse](data/psuedoinverse.png "OLS solution for parameter w")

In [ ]:
feature = PolynomialFeatures(degree=1)
X_train = feature.fit_transform(x_train)
X_train

In [ ]:
w = np.linalg.inv(X_train.transpose().dot(X_train)).dot(X_train.transpose()).dot(y_train)
w

The true generating function was $1.17 + 3.14*x$.  So not too bad, but not right on.  WHY?

Let's slow down and do that again so we can see the linear algebra in action

In [ ]:
#w = np.linalg.inv(X_train.transpose().dot(X_train)).dot(X_train.transpose()).dot(y)
X_train.transpose()

In [ ]:
#w = np.linalg.inv(X_train.transpose().dot(X_train)).dot(X_train.transpose()).dot(y)
X_train.transpose().dot(X_train)

In [ ]:
#w = np.linalg.inv(X_train.transpose().dot(X_train)).dot(X_train.transpose()).dot(y)
np.linalg.inv(X_train.transpose().dot(X_train))

In [ ]:
#w = np.linalg.inv(X_train.transpose().dot(X_train)).dot(X_train.transpose()).dot(y)
np.linalg.inv(X_train.transpose().dot(X_train)).dot(X_train.transpose())

In [ ]:
np.linalg.inv(X_train.transpose().dot(X_train)).dot(X_train.transpose()).dot(y_train)

# There has to be an easier way!
There is! __scikit-learn__ provides a uniform interface to dozens of machine learning algorithms.  They code up the math, and make the computation of the math effecient/fast.

In [ ]:
# if we hadn't have already built the bias term into features, you need to call fit_intercept = True!!!
model = LinearRegression(fit_intercept=False) 
model.fit(X_train, y_train)
model.coef_

# OK, 1 last thing
What if we try to do higher order curve fitting to data which is fundamentally linear?

In [ ]:
# make a graph with 2x2 subplots
fig, axes = plt.subplots(2,2,sharex=True, sharey=True,figsize=(10, 8))
axs = axes.flatten()

# loop through fitting/plotting 0th, 1st, 3rd, and 9th order polynomials
for i, degree in enumerate([ 0, 1, 3, 9]):
    ax = axs[i]
    feature = PolynomialFeatures(degree)
    X_train = feature.fit_transform(x_train)
    X_predict = feature.fit_transform(x_predict)

    model = LinearRegression(fit_intercept=False)
    model.fit(X_train, y_train)
    y_predict = model.predict(X_predict)

    ax.scatter(x_train, y_train, facecolor="none", edgecolor="b", s=50, label="training data")
    ax.plot(x_predict, y_true, c="g", label="true function")
    ax.plot(x_predict, y_predict, c="r", label="fitted curve")
    ax.annotate("M={}".format(degree), xy=(.15, .05),  xycoords='axes fraction', fontsize=14)
plt.legend(bbox_to_anchor=(1.05, 0.64), loc=2, borderaxespad=0.)
plt.suptitle('Linear regression with 10 training data points',fontsize=20)
plt.show()

Hey wait? Did we just use OLS ("linear") regression to fit a non-linear curve to data??? **Why did that work?**